# "Dynamic Temporal Modeling of Abdominal Aortic Aneurysm Morphology with Z-SINDy"

Authors: Joseph A. Pugar, Junsung Kim, Michael Mansour, Nhung Nguyen, C. J. Lee, Hence Verhagen, Ross Milner, Andrei A. Klishin, and Luka Pocivavsek

Code organized by J.P. and M.M.

# 0. Libraries, plotting styles, `.py` files

In [ ]:
import os
import re
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import numpy as np
import pandas as pd
from math import comb
from tqdm import tqdm
from numpy.polynomial.polynomial import polyfit, polyval

import matplotlib
from matplotlib import rcParams, gridspec
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.ticker import AutoMinorLocator, MaxNLocator
from matplotlib.collections import PolyCollection
from matplotlib.patches import Rectangle
from matplotlib.ticker import FixedLocator, FuncFormatter
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from mpl_toolkits.mplot3d import Axes3D

import zsindy as zsindy
from zsindy.dynamical_models import DynamicalSystem
from zsindy.ml_module import ZSindy

from scipy.integrate import solve_ivp
from scipy.spatial import ConvexHull, Delaunay
from scipy.interpolate import griddata
from scipy.stats import gaussian_kde
from scipy.special import logsumexp

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [ ]:
rcParams['mathtext.fontset']='cm'
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14 
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14

In [ ]:
from aaa_org_helpers import *
from aaa_zsindy import *
from graphical_abstract import *
from aaa_vector_fields import *
from aaa_bayesclf import *
from aaa_attrition import *

# 1. Preprocessing

## 1.0 Data import, organization, normalization, and FEA upsampling

In [ ]:
xvar = 'SurfaceArea'
yvar = 'IntGaussian_Fluct'

input_data_dict = build_all_norm_compressed(
    path_to_data=r'C:\Users\jpugar\Dropbox\Chicago\Papers\aaa-dynamics\data',
    xvar=xvar,
    yvar=yvar,
    path_sheet_names=['UChicago', 'ErasmusU'],
    fea_sheet_names=['UChicago_Regr_FEA', 'ErasmusU_Regr_FEA',
                     'UChicago_Stable_FEA', 'ErasmusU_Stable_FEA']
)

NonPath = input_data_dict['NonPath']
NonPath['Patient_ID'] = NonPath['Scan_ID'].str.split('_', n=1).str[0]

UChicago = input_data_dict['UChicago']
UChicago['Patient_ID'] = UChicago['Scan_ID'].str.split('_', n=1).str[0]
UChicago['Label_mean'] = UChicago['Label_mean'].astype(float)
a = len(UChicago['Patient_ID'].unique())
UChicago = UChicago[UChicago['Label_mean'].isin([1.0, 3.0])] # Ensure just regressing and stable outcomes 

ErasmusU = input_data_dict['ErasmusU']
ErasmusU['Patient_ID'] = ErasmusU['Scan_ID'].str.split('_', n=1).str[0]
ErasmusU['Label_mean'] = ErasmusU['Label_mean'].astype(float)

print("Total U. Chicago Patients (Stable + Regressing):", len(UChicago['Patient_ID'].unique()))
print("Total Erasmus U. Patients (Stable + Regressing):", len(ErasmusU['Patient_ID'].unique()))
print("U. Chicago Patients removed because of alternative clinical outcome (e.g., Endoleak):", a - len(UChicago['Patient_ID'].unique()))

In [ ]:
avg_yr_interval(UChicago)

In [ ]:
avg_yr_interval(ErasmusU)

Average interval between consecutive scans (years) for regressing dataset: 2.66

Average interval between consecutive scans (years) for stable dataset: 1.37

In [ ]:
fea_down_regr = sample_fea_results_via_poly(input_data_dict, dt=2.66/10, N=40, order=3)

In [ ]:
fea_down_stable = sample_fea_results_via_poly(input_data_dict, dt=1.37/10, N=40, order=3)

In [ ]:
UChicago_RF = fea_down_regr.get('UChicago_Regr_FEA', input_data_dict['UChicago_Regr_FEA'])
ErasmusU_RF = fea_down_regr.get('ErasmusU_Regr_FEA', input_data_dict['ErasmusU_Regr_FEA'])
UChicago_SF = fea_down_stable.get('UChicago_Stable_FEA', input_data_dict['UChicago_Stable_FEA'])
ErasmusU_SF = fea_down_stable.get('ErasmusU_Stable_FEA', input_data_dict['ErasmusU_Stable_FEA'])

In [ ]:
plot_data = pd.concat([NonPath, UChicago, ErasmusU], axis=0)
plot_data_FEA = pd.concat([UChicago_RF, ErasmusU_RF, UChicago_SF, ErasmusU_SF], axis=0)
# We can only collect preoperative coordinates on the U. Chicago group because the Erasmus U. cohort only have 1 month and 1 year postop scans.
UChicago_pre = UChicago[UChicago['Op'] == 'pre']

## 1.1 Finite differences, outlier removal, and initial conditions from preoperative data

In [ ]:
FEA = plot_data_FEA.copy()

dFEA = compute_finite_differences(FEA)
dUChicago = compute_finite_differences(UChicago, verbose=True)
report_patient_loss(UChicago, dUChicago, "UChicago Regressing")
dErasmusU = compute_finite_differences(ErasmusU, verbose=True)
report_patient_loss(ErasmusU, dErasmusU, "ErasmusU Regressing")

dUChicago['Label_mean'] = round(dUChicago['Label_mean'], 0)
dUChicago_regr = dUChicago[dUChicago['Label_mean'] == 1.0]
print("Regressing Patients UChicago:", len(dUChicago_regr['Patient_ID'].unique()))
dUChicago_stable = dUChicago[dUChicago['Label_mean'] == 3.0]
print("Stable Patients UChicago:", len(dUChicago_stable['Patient_ID'].unique()))

dErasmusU['Label_mean'] = round(dErasmusU['Label_mean'], 0)
dErasmusU_regr = dErasmusU[dErasmusU['Label_mean'] == 1.0]
print("Regressing Patients ErasmusU:", len(dErasmusU_regr['Patient_ID'].unique()))
dErasmusU_stable = dErasmusU[dErasmusU['Label_mean'] == 3.0]
print("Stable Patients ErasmusU:", len(dErasmusU_stable['Patient_ID'].unique()))

dFEA['Label_mean'] = round(dFEA['Label_mean'], 0)
dFEA_regr = dFEA[dFEA['Label_mean'] == 1.0]
print("Regressing Patients FEA:", len(dFEA_regr['Patient_ID'].unique()))
dFEA_stable = dFEA[dFEA['Label_mean'] == 3.0]
print("Stable Patients FEA:", len(dFEA_stable['Patient_ID'].unique()))

In [ ]:
# The number of total scans in the datasets: 
len(UChicago) + len(ErasmusU)

In [ ]:
ErasmusU

In [ ]:
init_conds_regr = UChicago_pre[UChicago_pre['Label_mean'] == 1.0]
init_conds_stable = UChicago_pre[UChicago_pre['Label_mean'] == 3.0]

groups = {
    'All data': UChicago_pre,
    'Regressing': init_conds_regr,
    'Stable': init_conds_stable,
}

AK_summary = pd.DataFrame({name: summarize_AK(df) for name, df in groups.items()}).T
print(AK_summary)

## 1.2 Dataset Overview

In [ ]:
data_overview_regr = pd.concat([dUChicago_regr, dErasmusU_regr], axis=0)
data_overview_stable = pd.concat([dUChicago_stable, dErasmusU_stable], axis=0)

In [ ]:
plot_dataset_overview(data_overview_regr, data_overview_stable)

### 1.2.1 Figure 12

Temporal data plot to show clinical trial attrition.

In [ ]:
plot_data_attrition(data_overview_regr, data_overview_stable)

In [ ]:
print("Regressing Patients:", len(data_overview_regr['Patient_ID'].unique()))
print("Stable Patients:", len(data_overview_stable['Patient_ID'].unique()))
print("U. Chicago Patients:", len(UChicago['Patient_ID'].unique()))
print("Erasmus U. Patients:", len(ErasmusU['Patient_ID'].unique()))

### 1.2.2 Figure 2.B

Barcode plot to show data scarcity and attrition.

In [ ]:
plot_patient_barcode(data_overview_regr, data_overview_stable, dpi=600, save_path=None)

### 1.2.3 Figure 1

Visualization of specific patients within the normalized size-shape feature space.

In [ ]:
single_patient_trajectory(plot_data_FEA.copy(), 
                         highlight_ids=['JK67'], 
                         id_col='Patient_ID',
                         show_background=False, 
                         xlim=(0,5), ylim=(0, 8), 
                         figsize=(5.5, 5.5),
                         save=False, dpi=600)

# 2. Z-SINDy modeling

## 2.1 Regressing Sacs

In [ ]:
# Regressing
# Organizing all 7 unique dataset combinations

# 1) UChicago CT data only 
dUChicago_regr.sort_values('Years_mean', ascending=True) 

# 2) ErasmusU CT data only
dErasmusU_regr.sort_values('Years_mean', ascending=True)

# 3) Combined UChicago + ErasmusU CT data
regr_CT = pd.concat([dUChicago_regr, dErasmusU_regr], axis=0).sort_values('Years_mean', ascending=True)

# 4) UChicago FEA data only
dFEA_regr_UChicago = dFEA_regr[dFEA_regr['Patient_ID'].str.contains('JK')].sort_values('Years_mean', ascending=True)

# 5) ErasmusU FEA data only
dFEA_regr_ErasmusU = dFEA_regr[dFEA_regr['Patient_ID'].str.contains('EU')].sort_values('Years_mean', ascending=True)

# 6) Combined UChicago + ErasmusU FEA data
regr_FEA = pd.concat([dFEA_regr_UChicago, dFEA_regr_ErasmusU], axis=0).sort_values('Years_mean', ascending=True)

# 7) Combined CT + FEA data
regr_all = pd.concat([regr_CT, regr_FEA], axis=0).sort_values('Years_mean', ascending=True)

# Example: Mean time interval between scans for the regressing CT dataset
regr_CT['Delta_Years_mean'].mean()

In [ ]:
x = np.array(regr_all[['SurfaceArea_Norm_mean', 'IntGaussian_Fluct_Norm_mean']].values)
t = np.array(regr_all['Years_mean'].values)
xdot = np.array(regr_all[['dSurfaceArea_Norm_mean/dYears_mean', 'dIntGaussian_Fluct_Norm_mean/dYears_mean']].values)
mu, sigma, rho = estimate_rho(x, xdot)
lambdas = np.linspace(-10, 10, 100)
sindy_tuner(lambdas, rho, x, t, xdot)

In [ ]:
Afixed, Kfixed = get_fixed_points(mu)
print(f"Fixed points: A = {Afixed}, K = {Kfixed}")

In [ ]:
def get_jacobian_from_mu(mu):
    """
    Assume Theta = [1, A, K] and mu.shape == (3, 2)
    Columns: equation for A' and K'
    Rows:    constant, A, K terms (in that order)
    """
    J = np.array([[mu[1, 0], mu[2, 0]],   # dA/dt coefficients on [A, K]
                  [mu[1, 1], mu[2, 1]]])  # dK/dt coefficients on [A, K]
    return J

def eig_timescales(mu):
    J = get_jacobian_from_mu(mu)
    eigvals, eigvecs = np.linalg.eig(J)

    # characteristic timescales (same units as your t, e.g. years)
    # use abs(real) so you get positive decay/growth times
    tau = 1.0 / np.abs(np.real(eigvals))

    return eigvals, eigvecs, tau

In [ ]:
eigvals, eigvecs, tau = eig_timescales(mu)
print("Eigenvalues:", eigvals)
print("Characteristic timescales (1/|Re λ|):", tau)

In [ ]:
zmodel = ZSindy(poly_degree=1, 
                 lmbda=-5, 
                 max_num_terms=3, 
                 rho=rho,
                 variable_names=['A', 'K'])
zmodel.fit(x, t, xdot)
z_xdot_pred = zmodel.predict()
zmodel.print()

In [ ]:
regr_i = 1 # index for initial conditions: 0=All data, 1=Regressing, 2=Stable
x0 = [AK_summary.iloc[regr_i,0], AK_summary.iloc[regr_i,2]]
coef = zmodel.coefficients()
std_coef = np.sqrt(zmodel.coefficients_variance())
t_span = (t[0], 10)
t_eval = np.linspace(*t_span, 1000)
sol = solve_ivp(DiffEq, t_span, x0, t_eval=t_eval, args=(coef,))
x0_std = [AK_summary.iloc[regr_i,1]/3, AK_summary.iloc[regr_i,3]/3]
z_ensemble_trials = 1000
num_feats = zmodel.coefficients().shape[1]
num_dims = x.shape[1]
z_coef_ensemble = np.zeros((z_ensemble_trials, num_dims, num_feats))
for i in tqdm(range(z_ensemble_trials)):
    for j in range(num_dims):
        z_coef_ensemble[i, j, :] = np.random.normal(zmodel.coefficients()[j, :], std_coef[j])
z_xdot_pred_ensemble = np.zeros((z_ensemble_trials, len(t), num_dims))
z_xpred_ensemble = np.zeros((z_ensemble_trials, len(t), num_dims))
print(f"\nSimulating ensemble of {z_ensemble_trials} Z-Sindy models...")

for i in tqdm(range(z_ensemble_trials)):
    x0_sample = np.random.normal(x0, x0_std)
    z_xdot_pred_ensemble[i, :, :] = zmodel.Theta @ z_coef_ensemble[i, :, :].T
    z_xpred_ensemble[i, :, :] = zmodel.simulate(x0_sample, t, coefs=z_coef_ensemble[i, :, :])

### 2.1.1 Figure 3.A

In [ ]:
sindy_subplot_regr(sol,
                   np.array(regr_CT[['SurfaceArea_Norm_mean', 'IntGaussian_Fluct_Norm_mean']].values), 
                   np.array(regr_CT['Years_mean'].values), 
                   t, 
                   np.array(regr_CT[['dSurfaceArea_Norm_mean/dYears_mean', 'dIntGaussian_Fluct_Norm_mean/dYears_mean']].values), 
                   z_xdot_pred, 
                   Afixed, 
                   Kfixed, 
                   size_color='#1f77b4', 
                   shape_color='#1f77b4',
                   z_xpred_ensemble=z_xpred_ensemble,
                   dpi=600)

### 2.1.2 Figure 9.A

In [ ]:
fig, ax = zsindy_coeff_distribution_plot(
    x, t, xdot, rho,
    n_splits=10, train_frac=0.8,
    lmbda=-5,
    poly_degree=1,
    max_num_terms=3,
    variable_names=('$\widetilde{A}$','$\widetilde{\delta K}$'),
    random_state=0,
    segment_color="#5edfff",
    point_color='#1f77b4',
    fig_size=(4,3),
    dpi=600
)
plt.show()

## 2.2 Stable Sacs

In [ ]:
# Stable
# Organizing all 7 unique dataset combinations
    
# 1) UChicago CT data only 
dUChicago_stable.sort_values('Years_mean', ascending=True) 

# 2) ErasmusU CT data only
dErasmusU_stable.sort_values('Years_mean', ascending=True)

# 3) Combined UChicago + ErasmusU CT data
stable_CT = pd.concat([dUChicago_stable, dErasmusU_stable], axis=0).sort_values('Years_mean', ascending=True)
        
# 4) UChicago FEA data only
dFEA_stable_UChicago = dFEA_stable[dFEA_stable['Patient_ID'].str.contains('JK')].sort_values('Years_mean', ascending=True)

# 5) ErasmusU FEA data only
dFEA_stable_ErasmusU = dFEA_stable[dFEA_stable['Patient_ID'].str.contains('EU')].sort_values('Years_mean', ascending=True)

# 6) Combined UChicago + ErasmusU FEA data
stable_FEA = pd.concat([dFEA_stable_UChicago, dFEA_stable_ErasmusU], axis=0).sort_values('Years_mean', ascending=True)

# 7) Combined CT + FEA data
stable_all = pd.concat([stable_CT, stable_FEA], axis=0).sort_values('Years_mean', ascending=True)

# Mean time interval between scans for the regressing CT dataset
stable_CT['Delta_Years_mean'].mean()

In [ ]:
x = np.array(stable_all[['SurfaceArea_Norm_mean', 'IntGaussian_Fluct_Norm_mean']].values)
t = np.array(stable_all['Years_mean'].values)
xdot = np.array(stable_all[['dSurfaceArea_Norm_mean/dYears_mean', 'dIntGaussian_Fluct_Norm_mean/dYears_mean']].values)
mu, sigma, rho = estimate_rho(x, xdot)
sindy_tuner(lambdas, rho, x, t, xdot)

In [ ]:
Afixed, Kfixed = get_fixed_points(mu)
print(f"Fixed points: A = {Afixed}, K = {Kfixed}")

In [ ]:
eigvals, eigvecs, tau = eig_timescales(mu)
print("Eigenvalues:", eigvals)
print("Characteristic timescales (1/|Re λ|):", tau)

In [ ]:
zmodel = ZSindy(poly_degree=1, 
                 lmbda=-5, 
                 max_num_terms=3, 
                 rho=rho,
                 variable_names=['A', 'K'])
zmodel.fit(x, t, xdot)
z_xdot_pred = zmodel.predict()
zmodel.print()

In [ ]:
stable_i = 2 # index for initial conditions: 0=All data, 1=stableessing, 2=Stable
x0 = [AK_summary.iloc[stable_i,0], AK_summary.iloc[stable_i,2]]
coef = zmodel.coefficients()
std_coef = np.sqrt(zmodel.coefficients_variance())
#t_span = (t[0], 10)
t_span = (-4, 10)
t_eval = np.linspace(*t_span, 1000)
sol = solve_ivp(DiffEq, t_span, x0, t_eval=t_eval, args=(coef,))
x0_std = [AK_summary.iloc[stable_i,1]/3, AK_summary.iloc[stable_i,3]/3]
z_ensemble_trials = 1000
num_feats = zmodel.coefficients().shape[1]
num_dims = x.shape[1]
z_coef_ensemble = np.zeros((z_ensemble_trials, num_dims, num_feats))
for i in tqdm(range(z_ensemble_trials)):
    for j in range(num_dims):
        z_coef_ensemble[i, j, :] = np.random.normal(zmodel.coefficients()[j, :], std_coef[j])
z_xdot_pred_ensemble = np.zeros((z_ensemble_trials, len(t), num_dims))
z_xpred_ensemble = np.zeros((z_ensemble_trials, len(t), num_dims))
print(f"\nSimulating ensemble of {z_ensemble_trials} Z-Sindy models...")

for i in tqdm(range(z_ensemble_trials)):
    x0_sample = np.random.normal(x0, x0_std)
    z_xdot_pred_ensemble[i, :, :] = zmodel.Theta @ z_coef_ensemble[i, :, :].T
    z_xpred_ensemble[i, :, :] = zmodel.simulate(x0_sample, t, coefs=z_coef_ensemble[i, :, :])

### 2.2.1 Figure 3.B

In [ ]:
sindy_subplot_stable(sol, 
                     np.array(stable_CT[['SurfaceArea_Norm_mean', 'IntGaussian_Fluct_Norm_mean']].values), 
                     np.array(stable_CT['Years_mean'].values), 
                     t, 
                     np.array(stable_CT[['dSurfaceArea_Norm_mean/dYears_mean', 'dIntGaussian_Fluct_Norm_mean/dYears_mean']].values), 
                     z_xdot_pred, 
                     Afixed, 
                     Kfixed, 
                     size_color='#d62728', 
                     shape_color='#d62728',
                     z_xpred_ensemble=z_xpred_ensemble,
                     dpi=600)

### 2.2.2 Figure 9.B

In [ ]:
fig, ax = zsindy_coeff_distribution_plot(
    x, t, xdot, rho,
    n_splits=10, train_frac=0.8,
    lmbda=-5,
    poly_degree=1,
    max_num_terms=3,
    variable_names=('$\widetilde{A}$','$\widetilde{\delta K}$'),
    random_state=0,
    segment_color="#f57b81",
    point_color='#d62728',
    fig_size=(4,3),
    dpi=600
)
plt.show()

## 2.3 Running the Z-SINDy pipeline for both the CT and CT+FEA datasets

In [ ]:
results_CT, regr_ens_CT, stable_ens_CT = zsindy_pipeline([{'df': regr_CT, 'name': '', 'lmbda': -5},],
                                                         [{'df': stable_CT, 'name': '', 'lmbda': -5},],
                                                         x0 = [AK_summary.iloc[0,0], AK_summary.iloc[0,2]],
                                                         x0_std = [AK_summary.iloc[0,1]/3, AK_summary.iloc[0,3]/3],
                                                         colors=colors,
                                                         plot_bool=False)

In [ ]:
results_all, regr_ens_all, stable_ens_all = zsindy_pipeline([{'df': regr_all, 'name': '', 'lmbda': -5},],
                                                         [{'df': stable_all, 'name': '', 'lmbda': -5},],
                                                         x0 = [AK_summary.iloc[0,0], AK_summary.iloc[0,2]],
                                                         x0_std = [AK_summary.iloc[0,1]/3, AK_summary.iloc[0,3]/3],
                                                         colors=colors,
                                                         plot_bool=False)

## 2.4 Graphical Abstract

In [ ]:
fig, ax, Traj_r, Traj_s = graphical_abstract(
    results_all,
    NonPath,
    n_traj_per_cohort=300,
    t_end=5.0,
    time_slices=(1.0, 2.5, 5.0),
    colors=(colors['Regr'], colors['Stable']),
    elev=10,
    azim=-115,
    dpi=600
)
style_3d_axis_tnr(ax, label_fs=22, tick_fs=18)
plt.tight_layout()
plt.show()

# 3. Vector flow fields

## 3.1 Figure 4: Empirical vectors, streamline flow field, Z-SINDy model output

In [ ]:
plot_stream_basic(
    df_regr   = results_CT[0]['df'],
    df_stable = results_CT[1]['df'],
    mu_regr   = results_all[0]['mu'],
    mu_stab   = results_all[1]['mu'],
    B_regr    = results_all[0]['B'],
    B_stab    = results_all[1]['B'],
    a_col="SurfaceArea_Norm_mean",
    k_col="IntGaussian_Fluct_Norm_mean",
    id_col="Patient_ID",
    time_col="Years_mean",
    step_size=0.05,
    arrowsize=1.0,
    quiver_scale=3.0,
    quiver_width=0.007,
    raw_blue="#1f77b4",
    raw_red ="#d62728",
    A_max=6.0,
    K_max=10.0,
    interp_smoothing=1e2,
    smoothing_kernel='thin_plate_spline',
    dpi=600
)

In [ ]:
out_regr = eig_and_times_from_mu(results_all[0]['mu'])
out_stab = eig_and_times_from_mu(results_all[1]['mu'])

print("REGRESSING model")
print("x* =", out_regr["x_star"])
print("eigenvalues =", out_regr["eigenvalues"])
print("taus (years) =", out_regr["taus"])
print("eigenvectors (columns) =\n", out_regr["eigenvectors"])

print("\nSTABLE model")
print("x* =", out_stab["x_star"])
print("eigenvalues =", out_stab["eigenvalues"])
print("taus (years) =", out_stab["taus"])
print("eigenvectors (columns) =\n", out_stab["eigenvectors"])


### 3.1.1 Justification for the `interp_smoothing` parameter

#### Figure 10

In [ ]:
fig_smooth, ax_smooth, smooth_results = justify_interp_smoothing_rbf(
    df_regr   = results_CT[0]['df'],
    df_stable = results_CT[1]['df'],
    a_col="SurfaceArea_Norm_mean",
    k_col="IntGaussian_Fluct_Norm_mean",
    id_col="Patient_ID",
    time_col="Years_mean",
    smoothing_values=np.logspace(-5, 5, 30),
    kernel='thin_plate_spline',
    chosen_smoothing=1e2,
    raw_blue="#1f77b4",
    raw_red ="#d62728",
    dpi=600
)

###  3.1.2 Flow field analysis for Z-SINDy models built from 2nd and 3rd order polynomial libraries

#### 3.1.2.1 2nd order system

In [ ]:
results_all_2, regr_ens_2, stable_ens_2 = zsindy_pipeline_2order(
    [{'df': regr_all,   'name': '', 'lmbda': -1e10}],
    [{'df': stable_all, 'name': '', 'lmbda': -1e10}],
    max_terms=6,
    x0=[AK_summary.iloc[0,0], AK_summary.iloc[0,2]],
    x0_std=[AK_summary.iloc[0,1]/3, AK_summary.iloc[0,3]/3],
    ens_trials=500,
    colors=colors,
    plot_bool=False,
)

#### Figure 8.A

In [ ]:
fig_adv, axes_adv = plot_stream_advanced(
    df_regr   = results_CT[0]['df'],
    df_stable = results_CT[1]['df'],
    model_regr = results_all_2[0]['model'],
    model_stab = results_all_2[1]['model'],
    a_col="SurfaceArea_Norm_mean",
    k_col="IntGaussian_Fluct_Norm_mean",
    step_size=0.05,
    arrowsize=1.0,
    raw_blue="#1f77b4",
    raw_red ="#d62728",
    A_max=6.0,
    K_max=10.0,
    dpi=600
)

#### 3.1.2.2 3rd order system

In [ ]:
results_all_3, regr_ens_3, stable_ens_3 = zsindy_pipeline_3order(
    [{'df': regr_all,   'name': '', 'lmbda': -1e10}],
    [{'df': stable_all, 'name': '', 'lmbda': -1e10}],
    max_terms=10,
    x0=[AK_summary.iloc[0,0], AK_summary.iloc[0,2]],
    x0_std=[AK_summary.iloc[0,1]/3, AK_summary.iloc[0,3]/3],
    ens_trials=100,
    colors=colors,
    plot_bool=False,
)

#### Figure 8.B

In [ ]:
fig_adv, axes_adv = plot_stream_advanced(
    df_regr   = results_CT[0]['df'],
    df_stable = results_CT[1]['df'],
    model_regr = results_all_3[0]['model'],
    model_stab = results_all_3[1]['model'],
    a_col="SurfaceArea_Norm_mean",
    k_col="IntGaussian_Fluct_Norm_mean",
    step_size=0.05,
    arrowsize=1.0,
    raw_blue="#1f77b4",
    raw_red ="#d62728",
    A_max=6.0,
    K_max=10.0,
    dpi=600
)

# 4. Bayesian classifiers

In [ ]:
datasets_regr = [
  {'df': regr_CT, 'name': '', 'lmbda': -5},
  {'df': regr_all, 'name': '', 'lmbda': -5},
]

datasets_stable = [
  {'df': stable_CT, 'name': '', 'lmbda': -5},
  {'df': stable_all, 'name': '', 'lmbda': -5},
]

## 4.1 Static Bayesian Classifier

Using the history of available spatial coordinates (size-shape) to make a classification between regions of the phase space associated with regressing versus stable morphologies.

### 4.1.1 Static Bayes classifier on CT dataset

#### Figure 11.A

In [ ]:
data_r_static = datasets_regr[0]['df']
data_s_static = datasets_stable[0]['df']

df0_r_static = data_r_static.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Regr')
df0_s_static = data_s_static.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Stable')
df0_static = pd.concat([df0_r_static, df0_s_static], ignore_index=True).drop_duplicates('Patient_ID')
df0_static['p_c2'] = 0.5

timepoints = [1, 2, 5]

kde_regr_dict   = make_kde_dict_cumulative_times(data_r_static, timepoints)
kde_stable_dict = make_kde_dict_cumulative_times(data_s_static, timepoints)

dfs_static = [df0_static]
for t in timepoints:
    df_t = static_bayes_class(
        regr_kde   = kde_regr_dict[t],
        stable_kde = kde_stable_dict[t],
        regr_df    = data_r_static,
        stable_df  = data_s_static,
        t_min      = 0.0,
        t_max      = t,
        prior      = 0.5
    )
    dfs_static.append(df_t)

jitter_plot(data_r_static, data_s_static, timepoints, dfs_static, 
            acc_bounds=[0.1, 0.9], alpha_points=0.50, axis_title=r'$p(Stable\ Statics \mid Anatomic\ Derivatives)$', dpi=600)

### 4.1.2 Static Bayes classifier on CT+FEA dataset

#### Figure 11.B

In [ ]:
data_r_static = datasets_regr[1]['df']
data_s_static = datasets_stable[1]['df']

df0_r_static = data_r_static.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Regr')
df0_s_static = data_s_static.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Stable')
df0_static = pd.concat([df0_r_static, df0_s_static], ignore_index=True).drop_duplicates('Patient_ID')
df0_static['p_c2'] = 0.5

timepoints = [1, 2, 5]

kde_regr_dict   = make_kde_dict_cumulative_times(data_r_static, timepoints)
kde_stable_dict = make_kde_dict_cumulative_times(data_s_static, timepoints)

dfs_static = [df0_static]
for t in timepoints:
    df_t = static_bayes_class(
        regr_kde   = kde_regr_dict[t],
        stable_kde = kde_stable_dict[t],
        regr_df    = data_r_static,
        stable_df  = data_s_static,
        t_min      = 0.0,
        t_max      = t,
        prior      = 0.5
    )
    dfs_static.append(df_t)

jitter_plot(data_r_static, data_s_static, timepoints, dfs_static, 
            acc_bounds=[0.1, 0.9], alpha_points=0.50, axis_title=r'$p(Stable\ Statics \mid Anatomic\ Derivatives)$', dpi=600)

## 4.2 Dynamic Bayesian Classifier

Using the history of available spatial `rate` coordinates (dsize/dt-dshape/dt) in addition to spatial coordinates to make a classification between regressing versus stable morphologies.

### 4.2.1 Dynamic Bayes classifier for the CT dataset

#### Figure 11.C

In [ ]:
data_r = datasets_regr[0]['df']
data_s = datasets_stable[0]['df']
result = pd.concat([data_r, data_s], ignore_index=True)
df0_r = data_r.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Regr')
df0_s = data_s.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Stable')
df0   = pd.concat([df0_r, df0_s], ignore_index=True).drop_duplicates('Patient_ID')
df0['p_c2'] = 0.5
timepoints = [1, 2, 5]
dfs_dynamic = [df0]

for t in timepoints:
    df_dyn_t = dynamic_bayes_class(
        results = results_all,
        regr_df     = data_r,
        stable_df   = data_s,
        t_min       = 0.0,
        t_max       = t,
        prior       = 0.5,
        id_col      = 'Patient_ID',
        time_col    = 'Years_mean',
        verbose_fd  = False,
        static_kde_dicts = (kde_regr_dict, kde_stable_dict),
    )
    dfs_dynamic.append(df_dyn_t)

jitter_plot(data_r, data_s, timepoints, dfs_dynamic, 
            acc_bounds=[0.1, 0.9], alpha_points=0.50, axis_title=r'$p(Stable\ Dynamics \mid Anatomic\ Derivatives)$', dpi=600)

### 4.2.2 Dynamic Bayes classifier for the CT+FEA dataset

#### Figure 11.D

In [ ]:
data_r = datasets_regr[1]['df']
data_s = datasets_stable[1]['df']
result = pd.concat([data_r, data_s], ignore_index=True)
df0_r = data_r.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Regr')
df0_s = data_s.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Stable')
df0   = pd.concat([df0_r, df0_s], ignore_index=True).drop_duplicates('Patient_ID')
df0['p_c2'] = 0.5
timepoints = [1, 2, 5]
dfs_dynamic = [df0]

for t in timepoints:
    df_dyn_t = dynamic_bayes_class(
        results = results_all,
        regr_df     = data_r,
        stable_df   = data_s,
        t_min       = 0.0,
        t_max       = t,
        prior       = 0.5,
        id_col      = 'Patient_ID',
        time_col    = 'Years_mean',
        verbose_fd  = False,
        static_kde_dicts = (kde_regr_dict, kde_stable_dict),
    )
    dfs_dynamic.append(df_dyn_t)

jitter_plot(data_r, data_s, timepoints, dfs_dynamic, 
            acc_bounds=[0.1, 0.9], alpha_points=0.50, axis_title=r'$p(Stable\ Dynamics \mid Anatomic\ Derivatives)$', dpi=600)

## 4.3 Figure 5

A comprehensive figure showing the results of many synthetic Z-SINDy trajectories through the Bayesian classification scheme.

In [ ]:
regr_ens_all['Patient_ID'] = 'REGR' + regr_ens_all['trial'].astype(str)
stable_ens_all['Patient_ID'] = 'STABLE' + stable_ens_all['trial'].astype(str)
regr_ens_all = regr_ens_all.sort_values(['Patient_ID', 'Years_mean']).copy()
stable_ens_all = stable_ens_all.sort_values(['Patient_ID', 'Years_mean']).copy()
regr_ens_all['order'] = regr_ens_all.groupby('Patient_ID').cumcount() + 1
stable_ens_all['order'] = stable_ens_all.groupby('Patient_ID').cumcount() + 1
regr_ens_all['Scan_ID'] = regr_ens_all['Patient_ID'] + '_' + regr_ens_all['order'].astype(str)
stable_ens_all['Scan_ID'] = stable_ens_all['Patient_ID'] + '_' + stable_ens_all['order'].astype(str)

data_r_trans = stress_test_transformer(regr_ens_all)
data_s_trans = stress_test_transformer(stable_ens_all)
df0_r = data_r_trans.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Regr')
df0_s = data_s_trans.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Stable')
df0_static   = pd.concat([df0_r, df0_s], ignore_index=True).drop_duplicates('Patient_ID')
df0_static['p_c2'] = 0.5
df0_dyn = pd.concat([df0_r, df0_s], ignore_index=True).drop_duplicates('Patient_ID')
df0_dyn['p_c2'] = 0.5
timepoints = [1, 2, 5]
kde_regr_dict_trans   = make_kde_dict_cumulative_times(data_r_trans, timepoints)
kde_stable_dict_trans = make_kde_dict_cumulative_times(data_s_trans, timepoints)

In [ ]:
dfs_static = [df0_static]
for t in timepoints:
    df_t = static_bayes_class(
        regr_kde   = kde_regr_dict_trans[t],
        stable_kde = kde_stable_dict_trans[t],
        regr_df    = data_r_trans,
        stable_df  = data_s_trans,
        t_min      = 0.0,
        t_max      = t,
        prior      = 0.5,
    )
    dfs_static.append(df_t)

In [ ]:
dfs_dynamic = [df0_dyn]
for t in timepoints:
    df_dyn_t = dynamic_bayes_class(
        results          = results_all,
        regr_df          = data_r_trans,
        stable_df        = data_s_trans,
        t_min            = 0.0,
        t_max            = t,
        prior            = 0.5,
        id_col           = 'Patient_ID',
        time_col         = 'Years_mean',
        verbose_fd       = False,
        static_kde_dicts = (kde_regr_dict_trans, kde_stable_dict_trans),
    )
    dfs_dynamic.append(df_dyn_t)

In [ ]:
fig_results = SINDY_BAYES(
    datasets_regr=datasets_regr[1]['df'],
    datasets_stable=datasets_stable[1]['df'],
    dfs_static=dfs_static,
    dfs_dyn=dfs_dynamic,
    poly_degree=1,
    max_terms=3,
    ens_trials=1000,
    x0 = [AK_summary.iloc[0,0], AK_summary.iloc[0,2]],
    x0_std = [AK_summary.iloc[0,1]/3, AK_summary.iloc[0,3]/3],
    dpi=600
)

## 4.4 Figure 6

Perform stress tests on the dynamic classifier with the introduction of spatial and temporal noise.

In [ ]:
# This takes about 3 hours to run on a standard desktop
stress_test_results = run_stress_test_analysis(
    AK_summary,
    regr_all,
    stable_all,
    noise_list = (0.0, 0.1, 0.25),
    interval_list   = (1.0, None, None),
    int_range_list  = (None, (0.5, 1.5), (1.0, 3.0)),
    max_time        = 5.0,
    n_realizations  = 5)

In [ ]:
plot_classifier_curves(stress_test_results, figsize=(10,5), sd_mult=3.0, dpi=600)

## 4.5 Cross Validation Tables 

CV Tables for Static and Dynamic Classifiers

In [ ]:
def _compute_metrics(df_fold, low=0.1, high=0.9):
    df_valid = df_fold.dropna(subset=['p_c2', 'ground_truth'])
    n_total = len(df_valid)
    if n_total == 0:
        return np.nan, np.nan
    classified_mask = (df_valid['p_c2'] < low) | (df_valid['p_c2'] > high)
    correct_mask = (
        (df_valid['ground_truth'] == 'Stable') & (df_valid['p_c2'] > high)
    ) | (
        (df_valid['ground_truth'] == 'Regr') & (df_valid['p_c2'] < low)
    )
    n_classified = classified_mask.sum()
    n_correct    = (classified_mask & correct_mask).sum()
    acc = n_correct / n_classified if n_classified > 0 else np.nan
    classified_frac = n_classified / n_total
    return acc, classified_frac

### 4.5.1 Static classifier with CT data

In [ ]:
data_r_static = datasets_regr[0]['df']
data_s_static = datasets_stable[0]['df']

df0_r_static = data_r_static.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Regr')
df0_s_static = data_s_static.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Stable')
df0_static = pd.concat([df0_r_static, df0_s_static], ignore_index=True).drop_duplicates('Patient_ID')
df0_static['p_c2'] = 0.5

timepoints = [1, 2, 5]

kde_regr_dict   = make_kde_dict_cumulative_times(data_r_static, timepoints)
kde_stable_dict = make_kde_dict_cumulative_times(data_s_static, timepoints)

dfs_static = [df0_static]
for t in timepoints:
    df_t = static_bayes_class(
        regr_kde   = kde_regr_dict[t],
        stable_kde = kde_stable_dict[t],
        regr_df    = data_r_static,
        stable_df  = data_s_static,
        t_min      = 0.0,
        t_max      = t,
        prior      = 0.5
    )
    dfs_static.append(df_t)

kf = KFold(n_splits=5, shuffle=True, random_state=0)

cv_rows_static = []

for fold_idx, (_, test_index) in enumerate(kf.split(df0_static), start=1):
    test_ids = df0_static.iloc[test_index]['Patient_ID'].values

    for t, df_t in zip(timepoints, dfs_static[1:]):
        df_fold = df_t[df_t['Patient_ID'].isin(test_ids)].copy()
        acc, classified = _compute_metrics(df_fold, low=0.1, high=0.9)
        cv_rows_static.append({
            'fold': fold_idx,
            'timepoint': t,
            'accuracy_pct': acc * 100.0 if np.isfinite(acc) else np.nan,
            'classified_pct': classified * 100.0 if np.isfinite(classified) else np.nan,
        })
        
cv_results_static_df = pd.DataFrame(cv_rows_static)
cv_results_static_df

### 4.5.2 Static classifier with CT + FEA upsampling data

In [ ]:
data_r_static = datasets_regr[1]['df']
data_s_static = datasets_stable[1]['df']

df0_r_static = data_r_static.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Regr')
df0_s_static = data_s_static.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Stable')
df0_static = pd.concat([df0_r_static, df0_s_static], ignore_index=True).drop_duplicates('Patient_ID')
df0_static['p_c2'] = 0.5

timepoints = [1, 2, 5]

kde_regr_dict   = make_kde_dict_cumulative_times(data_r_static, timepoints)
kde_stable_dict = make_kde_dict_cumulative_times(data_s_static, timepoints)

dfs_static = [df0_static]
for t in timepoints:
    df_t = static_bayes_class(
        regr_kde   = kde_regr_dict[t],
        stable_kde = kde_stable_dict[t],
        regr_df    = data_r_static,
        stable_df  = data_s_static,
        t_min      = 0.0,
        t_max      = t,
        prior      = 0.5
    )
    dfs_static.append(df_t)

kf = KFold(n_splits=5, shuffle=True, random_state=0)

cv_rows_static = []

for fold_idx, (_, test_index) in enumerate(kf.split(df0_static), start=1):
    test_ids = df0_static.iloc[test_index]['Patient_ID'].values

    for t, df_t in zip(timepoints, dfs_static[1:]):
        df_fold = df_t[df_t['Patient_ID'].isin(test_ids)].copy()
        acc, classified = _compute_metrics(df_fold, low=0.1, high=0.9)
        cv_rows_static.append({
            'fold': fold_idx,
            'timepoint': t,
            'accuracy_pct': acc * 100.0 if np.isfinite(acc) else np.nan,
            'classified_pct': classified * 100.0 if np.isfinite(classified) else np.nan,
        })
        
cv_results_static_df = pd.DataFrame(cv_rows_static)
cv_results_static_df

### 4.5.3 Dynamic classifier with CT data

In [ ]:
data_r = datasets_regr[0]['df']
data_s = datasets_stable[0]['df']
result = pd.concat([data_r, data_s], ignore_index=True)
df0_r = data_r.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Regr')
df0_s = data_s.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Stable')
df0   = pd.concat([df0_r, df0_s], ignore_index=True).drop_duplicates('Patient_ID')
df0['p_c2'] = 0.5
timepoints = [1, 2, 5]
dfs_dynamic = [df0]

for t in timepoints:
    df_dyn_t = dynamic_bayes_class(
        results = results_all,
        regr_df     = data_r,
        stable_df   = data_s,
        t_min       = 0.0,
        t_max       = t,
        prior       = 0.5,
        id_col      = 'Patient_ID',
        time_col    = 'Years_mean',
        verbose_fd  = False,
        static_kde_dicts = (kde_regr_dict, kde_stable_dict),
    )
    dfs_dynamic.append(df_dyn_t)

kf = KFold(n_splits=5, shuffle=True, random_state=0)

cv_rows = []

for fold_idx, (_, test_index) in enumerate(kf.split(df0), start=1):
    test_ids = df0.iloc[test_index]['Patient_ID'].values

    for t, df_dyn_t in zip(timepoints, dfs_dynamic[1:]):
        df_fold = df_dyn_t[df_dyn_t['Patient_ID'].isin(test_ids)].copy()

        acc, classified = _compute_metrics(df_fold, low=0.1, high=0.9)

        cv_rows.append({
            'fold': fold_idx,
            'timepoint': t,
            'accuracy_pct': acc * 100.0 if np.isfinite(acc) else np.nan,
            'classified_pct': classified * 100.0 if np.isfinite(classified) else np.nan,
        })

cv_results_df = pd.DataFrame(cv_rows)
cv_results_df

### 4.5.4 Dynamic classifier with CT + FEA upsampling data

In [ ]:
data_r = datasets_regr[1]['df']
data_s = datasets_stable[1]['df']
result = pd.concat([data_r, data_s], ignore_index=True)
df0_r = data_r.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Regr')
df0_s = data_s.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Stable')
df0   = pd.concat([df0_r, df0_s], ignore_index=True).drop_duplicates('Patient_ID')
df0['p_c2'] = 0.5
timepoints = [1, 2, 5]
dfs_dynamic = [df0]

for t in timepoints:
    df_dyn_t = dynamic_bayes_class(
        results = results_all,
        regr_df     = data_r,
        stable_df   = data_s,
        t_min       = 0.0,
        t_max       = t,
        prior       = 0.5,
        id_col      = 'Patient_ID',
        time_col    = 'Years_mean',
        verbose_fd  = False,
        static_kde_dicts = (kde_regr_dict, kde_stable_dict),
    )
    dfs_dynamic.append(df_dyn_t)

kf = KFold(n_splits=5, shuffle=True, random_state=0)

cv_rows = []

for fold_idx, (_, test_index) in enumerate(kf.split(df0), start=1):
    test_ids = df0.iloc[test_index]['Patient_ID'].values

    for t, df_dyn_t in zip(timepoints, dfs_dynamic[1:]):
        df_fold = df_dyn_t[df_dyn_t['Patient_ID'].isin(test_ids)].copy()

        acc, classified = _compute_metrics(df_fold, low=0.1, high=0.9)

        cv_rows.append({
            'fold': fold_idx,
            'timepoint': t,
            'accuracy_pct': acc * 100.0 if np.isfinite(acc) else np.nan,
            'classified_pct': classified * 100.0 if np.isfinite(classified) else np.nan,
        })

cv_results_df = pd.DataFrame(cv_rows)
cv_results_df

# 5. Data attrition

In [ ]:
labels = ['Index\nEVAR', '1 Yr.', '2 Yrs.', '5 Yrs.']

In [ ]:
data_r = datasets_regr[0]['df']
data_s = datasets_stable[0]['df']
(_,_), (_,_), prop_points = plot_data_attrition_2groups(data_r, data_s, cat_1="Regr", cat_2="Stable",
                                                        color_1='#1f77b4', color_2='#d62728')

#### Figure 13.A

In [ ]:
regr_ens_all['Patient_ID'] = 'REGR' + regr_ens_all['trial'].astype(str)
stable_ens_all['Patient_ID'] = 'STABLE' + stable_ens_all['trial'].astype(str)
regr_ens_all = regr_ens_all.sort_values(['Patient_ID', 'Years_mean']).copy()
stable_ens_all = stable_ens_all.sort_values(['Patient_ID', 'Years_mean']).copy()
regr_ens_all['order'] = regr_ens_all.groupby('Patient_ID').cumcount() + 1
stable_ens_all['order'] = stable_ens_all.groupby('Patient_ID').cumcount() + 1
regr_ens_all['Scan_ID'] = regr_ens_all['Patient_ID'] + '_' + regr_ens_all['order'].astype(str)
stable_ens_all['Scan_ID'] = stable_ens_all['Patient_ID'] + '_' + stable_ens_all['order'].astype(str)

regr_ens_transformed = stress_test_transformer(
    regr_ens_all,
    add_noise=False,
    fixed_interval=1.0,
    keep_other_cols=True,
    random_interval_range=None,
)

stable_ens_transformed = stress_test_transformer(
    stable_ens_all,
    add_noise=False,
    fixed_interval=1.0,
    keep_other_cols=True,
    random_interval_range=None,    
)

timepoints = [1, 2, 5]
data_r_static = regr_ens_transformed
data_s_static = stable_ens_transformed
df0_r_static = data_r_static.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Regr')
df0_s_static = data_s_static.drop_duplicates('Patient_ID')[['Patient_ID']].assign(ground_truth='Stable')
df0_static   = pd.concat([df0_r_static, df0_s_static], ignore_index=True).drop_duplicates('Patient_ID')
df0_static['p_c2'] = 0.5
kde_regr_dict   = make_kde_dict_cumulative_times(regr_ens_transformed, timepoints)
kde_stable_dict = make_kde_dict_cumulative_times(stable_ens_transformed, timepoints)

dfs_static_pair, results_static_pair = static_classifier_plot(
        df0_static,
        kde_df1_dict=kde_regr_dict,
        kde_df2_dict=kde_stable_dict,
        data_df1_static=data_r_static,
        data_df2_static=data_s_static,
        timepoints=timepoints,
        labels=labels,
        cat1='Regr',
        color1=colors['Regr'],
        cat2='Stable',
        color2=colors['Stable'],
        prop_points=prop_points,
        attrition_mode='dropout',
        random_state=42,
        prior=(46/(46+29)),
        dpi=600
    )

#### Figure 13.B

In [ ]:
dfs_dyn, results_dyn = dynamic_classifier_plot(
    regr_ens_all,
    stable_ens_all,
    results_all,
    data_r_trans,
    data_s_trans,
    kde_regr_dict_trans,
    kde_stable_dict_trans,
    x0=x0,
    x0_std=x0_std,
    best_results_df1_df2=results_all,
    mode='preop',
    timepoints=timepoints,
    labels=labels,
    cat1="Regr",
    color1=colors["Regr"],
    cat2="Stable",
    color2=colors["Stable"],
    caps_label1="REGR",
    caps_label2="STABLE",
    prop_points=prop_points,
    attrition_mode='dropout',
    random_state=42,
    prior=(46/(46+29)),
    dpi=600      
)